In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import re
import numpy as np

### LOADING 
We load two dataframes : 

1- Fao_data : contains the information about food loose worldwide

2- food_categories : code of food groups according to FAO and its description

In [124]:
data = pd.read_csv("DATA/Fao_data.csv")
food_categories = pd.read_excel("DATA/food_categories.xlsx")

In [125]:
pd.options.display.max_columns = None

### FIRST CLEANING

In [126]:
data.shape

(25416, 18)

In [127]:
data.head(3)

,m49_code,country,region,cpc_code,commodity,year,loss_percentage,loss_percentage_original,loss_quantity,activity,food_supply_stage,treatment,cause_of_loss,sample_size,method_data_collection,reference,url,notes
0,104,Myanmar,NaN,23161.02,"Rice, milled",2015,1.78,1.78%,26.12kgs,Storage,Storage,"30 days storage, with trapping",Rodents,NaN,Controlled Experiment,"Dr Steven Belmain (2015), context post-harvest...",NaN,Reference has been generated automatically
1,104,Myanmar,NaN,23161.02,"Rice, milled",2015,11.77,11.77%,88.18kgs,Storage,Storage,"60 days storage, no trapping",Rodents,NaN,Controlled Experiment,"Dr Steven Belmain (2015), context post-harvest...",NaN,Reference has been generated automatically
2,104,Myanmar,NaN,23161.02,"Rice, milled",2015,5.88,5.88%,44.09kgs,Storage,Storage,"30 days storage, no trapping",Rodents,NaN,Controlled Experiment,"Dr Steven Belmain (2015), context post-harvest...",NaN,Reference has been generated automatically


Drop columns and rows that only contain NaN values if there is any

In [128]:
data = data.dropna( how= 'all', axis= 0)
data = data.dropna( how= 'all', axis= 1)
data.shape

(25416, 18)

In [129]:
data.isna().sum()

m49_code                        0
country                         0
region                      24202
cpc_code                        0
commodity                       0
year                            0
loss_percentage                 0
loss_percentage_original        0
loss_quantity               24877
activity                     2808
food_supply_stage            3391
treatment                   24096
cause_of_loss               24414
sample_size                 24224
method_data_collection        355
reference                   20303
url                          3293
notes                       23139
dtype: int64

Dop columns we are not going to use

In [130]:
data.drop(columns= ['m49_code', 'region', 'loss_percentage_original', 'loss_quantity', 'sample_size', 'method_data_collection', 'reference', 'url' , 'notes'], inplace = True)

In [131]:
data.head()

,country,cpc_code,commodity,year,loss_percentage,activity,food_supply_stage,treatment,cause_of_loss
0,Myanmar,23161.02,"Rice, milled",2015,1.78,Storage,Storage,"30 days storage, with trapping",Rodents
1,Myanmar,23161.02,"Rice, milled",2015,11.77,Storage,Storage,"60 days storage, no trapping",Rodents
2,Myanmar,23161.02,"Rice, milled",2015,5.88,Storage,Storage,"30 days storage, no trapping",Rodents
3,Myanmar,23161.02,"Rice, milled",2015,3.57,Storage,Storage,"60 days storage, with trapping",Rodents
4,Myanmar,23161.02,"Rice, milled",2015,17.65,Storage,Storage,"90 days storage, no trapping",Rodents


<u> SOME COLUMNS EXPLANATION </U>

**cpc_code** -- product category according to CPD classification. Each cpd is mutually exclusive

**loss percentage** --Interpreted as the percentage of production/commodity that does not reach the next stage of the Food Supply Chain. It is computed over the all produced and imported quantity of that commodity in the country.


In [132]:
data.dtypes

country               object
cpc_code              object
commodity             object
year                   int64
loss_percentage      float64
activity              object
food_supply_stage     object
treatment             object
cause_of_loss         object
dtype: object

### EXPLOARATION AND TRANSFORMATION

How many countries do we have information of, as well as how many commodities and CPC. Also how many stages

In [133]:
print(f"Number of countries = {len(data.country.unique())}\n"
      f"Number of commodities = {len(data.commodity.unique())}\n"
      f"Number of CPC = {len(data.cpc_code.unique())}\n"
      f"Number of stages = {len(data.activity.unique())}\n"
      )



Number of countries = 123
Number of commodities = 147
Number of CPC = 147
Number of stages = 128



As the 3 firts numbers of the cpd_code gives the GROPUP to which the commodity belongs we create a new column "group" holding this code.

In [134]:
# reducing CPC codes"
data['group'] = data['cpc_code'].str[:3]
data.head()

,country,cpc_code,commodity,year,loss_percentage,activity,food_supply_stage,treatment,cause_of_loss,group
0,Myanmar,23161.02,"Rice, milled",2015,1.78,Storage,Storage,"30 days storage, with trapping",Rodents,231
1,Myanmar,23161.02,"Rice, milled",2015,11.77,Storage,Storage,"60 days storage, no trapping",Rodents,231
2,Myanmar,23161.02,"Rice, milled",2015,5.88,Storage,Storage,"30 days storage, no trapping",Rodents,231
3,Myanmar,23161.02,"Rice, milled",2015,3.57,Storage,Storage,"60 days storage, with trapping",Rodents,231
4,Myanmar,23161.02,"Rice, milled",2015,17.65,Storage,Storage,"90 days storage, no trapping",Rodents,231


In [135]:
#the new column is created as a str and I want it as a integer 
data['group'] = data['group'].astype(int)

In [136]:
#we have been able to reduce categories
len(data.group.unique())

23

In [137]:
food_categories = pd.read_excel("DATA/food_categories.xlsx")

In [138]:
food_categories.head()

,Code,Group
0,11,Cereals
1,12,Vegetables
2,13,Fruit and nuts
3,14,Oilseeds and oleaginous fruits
4,15,Edible roots and tubers with high starch or in...


create a new column in the dataframe to have the gruoup description

In [139]:
# Merge dataframes based on the 'group' column
merged_df = pd.merge(data, food_categories, left_on='group', right_on='Code', how='left')

# Create a new column in data with the categories matching the 'group'
data['description'] = merged_df['Group']


In [140]:
data.sample(5)

,country,cpc_code,commodity,year,loss_percentage,activity,food_supply_stage,treatment,cause_of_loss,group,description
11881,Mali,0118,Millet,2010,0.471596,Winnowing,Farm,NaN,NaN,11,Cereals
15205,Nigeria,0114,Sorghum,2007,3.600000,"Shelling, Threshing",NaN,NaN,NaN,11,Cereals
15402,Pakistan,0114,Sorghum,2013,3.070000,Storage,Storage,F-114 variety incubated at 32 degrees celcius,NaN,11,Cereals
7657,India,01446,Safflower seed,2014,3.240000,NaN,Whole supply chain,NaN,Physical Losses,14,Oilseeds and oleaginous fruits
10307,Lesotho,0111,Wheat,2009,4.433330,"Drying, Harvesting",Harvest,NaN,NaN,11,Cereals


Checking the columns: 'acitvity', 'food_supply_stage','treatment' and 'cause of loss' to check the information

In [141]:
data.food_supply_stage.unique()

array(['Storage', 'Harvest', 'Farm', nan, 'Processing', 'Post-harvest',
       'Retail', 'Wholesale', 'Trader', 'Whole supply chain', 'Transport',
       'Households', 'Pre-harvest', 'Distribution', 'Food Services',
       'Export', 'Market', 'Collector', 'Stacking', 'Packing', 'Grading'],
      dtype=object)

 it has not lot of values so does not need further cleaning

In [142]:
data.activity.unique()

array(['Storage', 'Drying, Harvesting', 'Shelling, Threshing',
       'Transportation', 'Drying', 'Winnowing', 'Milling', 'Parboiling',
       nan, 'Farm', 'Retailing', 'Wholesale', 'Collection',
       'Farm, Marketing, Storage, Transportation', 'Processing, Ripening',
       'Consumption', 'Packaging, Processing', 'Handling, Storage',
       'Distribution', 'Harvesting',
       'Cleaning, Collection, Distribution, Drying, Field, Harvesting, Shelling, Storage, Threshing, Transportation',
       'Marketing', 'Drying, Farm', 'Processing', 'Handling',
       'Handling, Harvesting', 'Collection, Marketing',
       'Marketing, Wholesale', 'Harvesting, Sorting', 'Grading, Sorting',
       'Threshing', 'Sorting', 'Distribution, Retailing', 'Peeling',
       'Marketing, Storage', 'Distribution, Trading', 'Stacking',
       'Storage, Trading', 'Exporting', 'Manufacturing', 'Trading',
       'Farm, Threshing', 'Cleaning, Farm', 'Farm, Handling, Storage',
       'Handling, Trading, Transportatio

several activities in the same cell. Steps for cleaning:

 1- make a data frame where each activity corresponds to one column

2- see how many many columns we must keep in order to keep most the information without much complexity 
    

In [143]:
activities_df = data['activity'].str.split(', ', expand=True)
activities_df.head()

,0,1,2,3,4,5,6,7,8,9,10
0,Storage,None,None,None,None,None,None,None,None,None,None
1,Storage,None,None,None,None,None,None,None,None,None,None
2,Storage,None,None,None,None,None,None,None,None,None,None
3,Storage,None,None,None,None,None,None,None,None,None,None
4,Storage,None,None,None,None,None,None,None,None,None,None


In [144]:
activities_df.isna().sum()

0      2808
1     17804
2     25190
3     25236
4     25260
5     25265
6     25268
7     25270
8     25274
9     25275
10    25414
dtype: int64

In [145]:
#just keeping two columns we keep most of the information. 
mergeed_activities = pd.merge(data, activities_df, left_index=True, right_index=True, how='left')
mergeed_activities.head()

,country,cpc_code,commodity,year,loss_percentage,activity,food_supply_stage,treatment,cause_of_loss,group,description,0,1,2,3,4,5,6,7,8,9,10
0,Myanmar,23161.02,"Rice, milled",2015,1.78,Storage,Storage,"30 days storage, with trapping",Rodents,231,Grain mill products,Storage,None,None,None,None,None,None,None,None,None,None
1,Myanmar,23161.02,"Rice, milled",2015,11.77,Storage,Storage,"60 days storage, no trapping",Rodents,231,Grain mill products,Storage,None,None,None,None,None,None,None,None,None,None
2,Myanmar,23161.02,"Rice, milled",2015,5.88,Storage,Storage,"30 days storage, no trapping",Rodents,231,Grain mill products,Storage,None,None,None,None,None,None,None,None,None,None
3,Myanmar,23161.02,"Rice, milled",2015,3.57,Storage,Storage,"60 days storage, with trapping",Rodents,231,Grain mill products,Storage,None,None,None,None,None,None,None,None,None,None
4,Myanmar,23161.02,"Rice, milled",2015,17.65,Storage,Storage,"90 days storage, no trapping",Rodents,231,Grain mill products,Storage,None,None,None,None,None,None,None,None,None,None


In [146]:
data['activity_1'] = mergeed_activities[0]
data['activity_2'] = mergeed_activities[1]
data.sample(3)

,country,cpc_code,commodity,year,loss_percentage,activity,food_supply_stage,treatment,cause_of_loss,group,description,activity_1,activity_2
9402,Kenya,0114,Sorghum,2014,4.550,"Drying, Harvesting",Harvest,NaN,NaN,11,Cereals,Drying,Harvesting
8146,India,01329,"Other citrus fruit, n.e.c.",2006,0.920,Harvesting,Harvest,NaN,NaN,13,Fruit and nuts,Harvesting,None
24716,Zambia,0114,Sorghum,2020,4.444,"Drying, Harvesting",Harvest,NaN,NaN,11,Cereals,Drying,Harvesting


In [147]:
#once checked that we have the activites columns as we wanted we drop the original activity column
data.drop(columns= 'activity', inplace= True)

In [148]:
data.treatment.unique()

array(['30 days storage, with trapping', '60 days storage, no trapping',
       '30 days storage, no trapping', '60 days storage, with trapping',
       '90 days storage, no trapping', '90 days storage, with trapping',
       nan, 'Yard-long beans', 'Dry season 1 ( November to January)',
       'Wet season  ( May to October)',
       'Dry season 2 ( January to April)',
       'using improved handling and marketing practices',
       'using traditional practices for handling and  marketing',
       'Warehouse storage', 'Bags storage', 'Cabinet storage',
       'Other storage methods', 'Japonica rice',
       'Cabinet storage - Indica rice', 'Indica rice',
       'Other storage methods - Indica rice', 'Indica Rice',
       'Bags storage - Japonica rice', 'Warehouse storage - Indica rice',
       'Bags storage - Indica rice', 'Cabinet storage - Japonica rice',
       'Extreme conditions', 'Harvesting for fresh market',
       'Cabbage for storage', 'Harvesting for export', 'Trimming',
   

the column treatment is very noisy so it will be drop

In [149]:
data.drop(columns= 'treatment', inplace= True)

In [150]:
data.cause_of_loss.unique()[40:100]

array(['Physical Losses', 'Measured In May; Due To Fruit Flies',
       'Higher Losses In July Due To Fruit Flies',
       'In the field 1.7 59% 33%  During threshing 1.0 64% 26%  During transport 0.2 60% 26%  At germination 0.2 61% 22%  During temporary storage before  drying  0.1 58% 23%  During drying 0.9 60% 26%  During loading 0.1 57% 25%  During storage after drying 0.3 53% 29%  Other 7.2 22% 30%     Quality deficiencies  (e.g. protein and  starch quality) was  the main cause,  followed by plant  diseases, threshing  and weather  conditions.   Increased nitrogen  fertilisation, more  pesticide use, improved  storage, better cultivars  and reduction in  competitive  disadvantage.    Average (%)/ Frequency of producers/Frequency of producers who answered “Don’t know”   Disease in the field 6.3 38% 34%  Lodging (e.g. Caused by heavy rain) 2.4 43% 28%  Pest damage 2.8 40% 29%  Weather conditions (rain, floods, etc.) 3.7 40% 30%  During temporary storage before drying 0.6 35% 29%  Tec

In [151]:
data.cause_of_loss.isna().sum()

24414

In [152]:
words = ['rodents', 'insect', 'mechanical','whater', 'predators', 'trimming', 'handling', 'dropped', 'rat', 'spred', 'package', 'temperature', 'rain']

In [153]:
def extract_generic_loss_causes(text):
    """
    Extracts generic loss causes from a text.

    Parameters:
    text (str): The text containing potential loss causes.

    Returns:
    str: A string containing the extracted generic loss causes, separated by ', '.
    If no matches are found, returns an empty string.
    If the input text is NaN, returns NaN.
    """
    # Check if the value is NaN
    if pd.isnull(text):
        return np.nan
    
    # List of words to search for
    words = ['rodents', 'insect', 'mechanical', 'water', 'predators', 'trimming', 'handling', 
             'dropped', 'rat', 'spread', 'package', 'temperature', 'rain']
    
    # Regular expression pattern to match any word from the list
    pattern = r'\b(?:' + '|'.join(words) + r')\b'
    
    # Find all matching words in the text and convert them to lowercase
    matches = [match.lower() for match in re.findall(pattern, str(text), flags=re.IGNORECASE)]
    
    # Join the matching words into a single string separated by ', '
    result = ', '.join(matches)
    
    return result

In [154]:
# Apply the function to the 'cause_of_loss' column to create the new 'generic_loss_causes' column
data['generic_loss_causes'] = data['cause_of_loss'].apply(extract_generic_loss_causes)

Now we will do the same as we did with the 'activity' column to keep as much information with the minimum columns.

In [158]:
#check the NaN in the original column and in the new column 
print (f" Number of NaN in the 'cause_of_loss' column  = {data.cause_of_loss.isna().sum()}\n"
       f" Number of NaN in the 'generic_loss_causes' column  = {data.generic_loss_causes.isna().sum()}\n"
       f" difference between both  = {data.cause_of_loss.isna().sum() - data.generic_loss_causes.isna().sum()}"
       )

 Number of NaN in the 'cause_of_loss' column  = 24414
 Number of NaN in the 'generic_loss_causes' column  = 24414
 difference between both  = 0


it seems we have been able to get most of the valuable information as all the columns with 

In [157]:
data.generic_loss_causes.unique()

array(['rodents', nan, 'insect', '', 'handling', 'insect, rat',
       'mechanical', 'trimming', 'water, mechanical',
       'dropped, mechanical', 'spread, insect, handling', 'temperature',
       'rain, rain', 'rain, handling', 'rain', 'insect, rodents',
       'rodents, rain', 'handling, handling', 'handling, package',
       'insect, mechanical', 'water', 'predators',
       'handling, insect, handling, handling, insect', 'water, rain',
       'handling, insect, handling, trimming',
       'handling, mechanical, rodents, rodents, rodents, mechanical'],
      dtype=object)

In [159]:
loss_causes_df = data['generic_loss_causes'].str.split(', ', expand=True)
loss_causes_df.head()

,0,1,2,3,4,5
0,rodents,None,None,None,None,None
1,rodents,None,None,None,None,None
2,rodents,None,None,None,None,None
3,rodents,None,None,None,None,None
4,rodents,None,None,None,None,None


In [160]:
loss_causes_df.isna().sum()

0    24414
1    25373
2    25406
3    25409
4    25414
5    25415
dtype: int64

keeping just two columns we have almost all the information

In [162]:
data['cause_loss_1'] = loss_causes_df[0]
data['cause_loss_2'] = loss_causes_df[1]

In [163]:
data.head(3)

,country,cpc_code,commodity,year,loss_percentage,food_supply_stage,cause_of_loss,group,description,activity_1,activity_2,generic_loss_causes,cause_loss_1,cause_loss_2
0,Myanmar,23161.02,"Rice, milled",2015,1.78,Storage,Rodents,231,Grain mill products,Storage,None,rodents,rodents,None
1,Myanmar,23161.02,"Rice, milled",2015,11.77,Storage,Rodents,231,Grain mill products,Storage,None,rodents,rodents,None
2,Myanmar,23161.02,"Rice, milled",2015,5.88,Storage,Rodents,231,Grain mill products,Storage,None,rodents,rodents,None


### EXPORTATION

Once our dataset is cleaned and keeps all the relevant information we export it.

In [165]:
data.to_csv('DATA/clean_fao_informaton.csv',index= False)